# Compute the phonons and the Raman spectrum

In [1]:
from __future__ import print_function
from mybigdft.workflows import Phonons, RamanSpectrum
from mybigdft import Job, Posinp, Atom, InputParams

In [2]:
N2_ref = """2  angstroem
free
N    2.976307744763e-23    6.872205902435e-23    1.071620018790e-02
N   -1.104344885754e-23   -4.873421785298e-23    1.104273795769e+00"""
pos = Posinp.from_string(N2_ref)
gs = Job(posinp=pos, name='N2', run_dir='../../../tests/phonons_N2')
phonons = Phonons(gs)

In [3]:
phonons.run(nmpi=2, nomp=2)

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/phonons_N2/atom0000/x+
Data directory not found for reference calculation.
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/phonons_N2/atom0000/x-
Data directory not found for reference calculation.
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/phonons_N2/atom0000/y+
Data directory not found for reference calculation.
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/phonons_N2/atom0000/y-
Data directory not found for reference calculation.
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/phonons_N2/atom0000/z+
Data directory not found for reference calculation.
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/phonons_N2/atom0000/z-
Data directory not found for reference calculation.
Logfile log-N2.yaml already exists!

In [4]:
phonons.energies['cm^-1']

array([2.38632021e+03, 5.52930617e-06, 2.01915074e+01, 2.01915188e+01,
       3.08714641e-06, 2.76516701e-03])

In [5]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")  # This catches the UserWarning raised
    raman = RamanSpectrum(phonons)
    raman.run(nmpi=2, nomp=2)

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/phonons_N2/atom0000/x+
Data directory not found for reference calculation.
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/phonons_N2/atom0000/x+
Data directory not found for reference calculation.
Logfile log-N2_EF_along_x.yaml already exists!

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/phonons_N2/atom0000/x+
Data directory not found for reference calculation.
Logfile log-N2_EF_along_y.yaml already exists!

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/phonons_N2/atom0000/x+
Data directory not found for reference calculation.
Logfile log-N2_EF_along_z.yaml already exists!

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/phonons_N2/atom0000/x-
Data directory not found for reference calculation.
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/phonons_N2/atom0000/x-
Data directory not found for reference calculation.
Lo

In [6]:
raman.energies['cm^-1']

array([2.38632021e+03, 5.52930617e-06, 2.01915074e+01, 2.01915188e+01,
       3.08714641e-06, 2.76516701e-03])

In [7]:
raman.intensities

array([2.25618074e+01, 4.08907417e-23, 1.08444042e+00, 1.07725869e+00,
       1.42410782e-22, 3.03173050e-11])

In [8]:
raman.depolarization_ratios

array([0.09374867, 0.73033043, 0.75      , 0.75      , 0.14281021,
       0.09406008])